In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import numpy as np
import unicodedata
import string
import re
import random
import torchtext
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
from google.colab import drive
import nltk as nl
from tqdm import tqdm  
nl.download('punkt')
# drive.mount('/content/drive',force_remount=True)
from matplotlib import pyplot as plt
import datetime
import time
import copy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import gensim.downloader as api


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The implementation of Transformer is refered to [https://github.com/qingyujean/Magic-NLPer/tree/main/NLP/MachineTranslation/transformer/pytorch] and [https://tensorflow.google.cn/tutorials/text/transformer]. The decription for each part is follow the tutoral of tensorflow

In [ ]:
# tokenizer = lambda x: x.split() # word-level
tokenizer = lambda x: list(x) # char-level

In [ ]:
p_intent = []
p_snippet = []
c = 0
iit = []
with open('x.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      if i[:-1].lower() not in p_intent:
        iit.append(c)
      c+=1
      p_intent.append(i[:-1].lower())
p_intentn = []
with open('x.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      p_intentn.append(i[:-1].lower())
with open('y.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippet.append(i.lower())
  
train_pairs = [[p_intentn[i],p_snippet[i]] for i in iit]
p_intentval = []
p_snippetval = []
iiv = []
c= 0
with open('xv.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      if i[:-1].lower() not in p_intent and i[:-1].lower() not in p_intentval:
        iiv.append(c)
      
      p_intentval.append(i[:-1].lower())
      c+=1
p_intentvaln = []
with open('xv.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      p_intentvaln.append(i[:-1].lower())
      c+=1
with open('yv.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      #i = i.split(' ')
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippetval.append(i.lower())
val_pairs = [[p_intentvaln[i],p_snippetval[i]] for i in iiv]
p_intenttest = []
p_snippettest = []
ii = []
c= 0
with open('xt.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      if i[:-1].lower() not in p_intent and i[:-1].lower() not in p_intenttest:
        ii.append(c)
      c+=1
      p_intenttest.append(i[:-1].lower())
with open('yt.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      # if i.lower() not in p_snippet:
      p_snippettest.append(i.lower())
test_pairs = [[p_intenttest[i],p_snippettest[i]] for i in ii]
p_intenttest1 = []
p_snippettest1 = []
with open('xt1.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      
      p_intenttest1.append(i[:-1].lower())
with open('yt1.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippettest1.append(i.lower())
test_pairs1 = [[p_intenttest1[i],p_snippettest1[i]] for i in ii]
p_intenttest2 = []
p_snippettest2 = []
with open('xt2.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      
      p_intenttest2.append(i[:-1].lower())
with open('yt2.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippettest2.append(i.lower())
test_pairs2 = [[p_intenttest2[i],p_snippettest2[i]] for i in ii]

In [ ]:

SRC_TEXT = torchtext.legacy.data.Field(sequential=True,
                                tokenize=tokenizer,

                                preprocessing=lambda x: ['sos'] + x + ['eos'],
                                
                                )
TARG_TEXT = torchtext.legacy.data.Field(sequential=True,
                                 tokenize=tokenizer,

                                 preprocessing=lambda x: ['sos'] + x + ['eos'],
                                 )
                                 


def get_dataset(pairs, src, targ):
    fields = [('src', src), ('targ', targ)]  
    examples = []  
    for src, targ in tqdm(pairs): 
        examples.append(torchtext.legacy.data.Example.fromlist([src, targ], fields))
    return examples, fields


# examples, fields = get_dataset(pairs, SRC_TEXT, TARG_TEXT)

ds_train = torchtext.legacy.data.Dataset(*get_dataset(train_pairs, SRC_TEXT, TARG_TEXT))
ds_val = torchtext.legacy.data.Dataset(*get_dataset(val_pairs, SRC_TEXT, TARG_TEXT))
ds_test = torchtext.legacy.data.Dataset(*get_dataset(test_pairs, SRC_TEXT, TARG_TEXT))

ds_test1 = torchtext.legacy.data.Dataset(*get_dataset(test_pairs1, SRC_TEXT, TARG_TEXT))
ds_test2 = torchtext.legacy.data.Dataset(*get_dataset(test_pairs2, SRC_TEXT, TARG_TEXT))

100%|██████████| 16095/16095 [00:00<00:00, 42511.74it/s]


In [ ]:
SRC_TEXT.build_vocab(ds_train) 
print(len(SRC_TEXT.vocab))
print(SRC_TEXT.vocab.itos[0])
print(SRC_TEXT.vocab.itos[1])
print(SRC_TEXT.vocab.itos[2])
print(SRC_TEXT.vocab.itos[3])
print(SRC_TEXT.vocab.stoi['sos'])
print(SRC_TEXT.vocab.stoi['eos'])


TARG_TEXT.build_vocab(ds_train)
print(len(TARG_TEXT.vocab))
print(TARG_TEXT.vocab.itos[0])
print(TARG_TEXT.vocab.itos[1])
print(TARG_TEXT.vocab.itos[2])
print(TARG_TEXT.vocab.itos[3])
print(TARG_TEXT.vocab.stoi['sos'])
print(TARG_TEXT.vocab.stoi['eos'])


55
<unk>
<pad>
 
e
14
13
62
<unk>
<pad>
 
)
15
14


In [ ]:
class DataLoader:
    def __init__(self, data_iter):
        self.data_iter = data_iter
        self.length = len(data_iter)  

    def __len__(self):
        return self.length

    def __iter__(self):
        # switch to batch first
        for batch in self.data_iter:
            yield (torch.transpose(batch.src, 0, 1), torch.transpose(batch.targ, 0, 1))




## Positional encoding


In [ ]:
def get_angles(pos, i, d_model):
    # 2*(i//2) make sure that calculating 1/10000^(2i/d)
    angle_rates = 1 / np.power(10000, 2 * (i // 2) / np.float32(d_model))
    return pos * angle_rates  

def positional_encoding(position, d_model):  # d_model = embedding_dim
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],  
                            np.arange(d_model)[np.newaxis, :], 
                            d_model)
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  # 2i
    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])  # 2i+2
    pos_encoding = angle_rads[np.newaxis, ...]  
    return torch.tensor(pos_encoding, dtype=torch.float32)

## Masking
The padding mask indicates where is a padding token: it outputs a 1 at locations of padding tokens, and a 0 otherwise. It guarantee that the model does not treat padding as the input.

The look-ahead mask is used to mask the future tokens in a target sequence. In other words, the mask indicates which target tokens should not be used. For example, to predict the third target token, only the first and second target token will be allowed to use.

In [ ]:
pad = 1 # When building vocab in Pytorch, index 1 represent padding token 
def create_padding_mask(seq):  # seq [b, seq_len]
    seq = torch.eq(seq, torch.tensor(pad)).float()  # pad!=0
    return seq[:, np.newaxis, np.newaxis, :]  # =>[b, 1, 1, seq_len]
def create_look_ahead_mask(size):  # seq_len
    mask = torch.triu(torch.ones((size, size)), diagonal=1)
    return mask  # [seq_len, seq_len]
def create_mask(inp, targ):
    # encoder padding mask
    # Used in the self-attention in the encoder.
    enc_padding_mask = create_padding_mask(inp)  # =>[b,1,1,inp_seq_len] 
    
    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)  # =>[b,1,1,inp_seq_len] 
    
    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(targ.shape[-1])  # =>[targ_seq_len,targ_seq_len]
    dec_targ_padding_mask = create_padding_mask(targ)  # =>[b,1,1,targ_seq_len]
    combined_mask = torch.max(look_ahead_mask, dec_targ_padding_mask)  # combine 2 masks =>[b,1,targ_seq_len,targ_seq_len]

    return enc_padding_mask, combined_mask, dec_padding_mask

## Attention mechanism

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.

    Args:
      q: query shape == (..., seq_len_q, depth)
      k: key shape == (..., seq_len_k, depth)
      v: value shape == (..., seq_len_v, depth_v)
      mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
      output, attention_weights
    """
    
    matmul_qk = torch.matmul(q, k.transpose(-1, -2))  #[..., seq_len_q, seq_len_k]
    # scale matmul_qk
    dk = torch.tensor(k.shape[-1], dtype=torch.float32)  # depth of k
    scaled_attention_logits = matmul_qk / torch.sqrt(dk)  # [..., seq_len_q, seq_len_k]

    # add the mask to the scaled tensor.
    if mask is not None:  # mask: [batch_size, 1, 1, seq_len]
        # The mask is multiplied with -1e9 (close to negative infinity). 
        # This is done because the mask is summed with the scaled matrix 
        # multiplication of Q and K and is applied 
        # immediately before a softmax.
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = torch.nn.functional.softmax(scaled_attention_logits, dim=-1)  

    output = torch.matmul(attention_weights, v)  
    return output, attention_weights  

In [ ]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0  #need to make sure each head have same dimension

        self.depth = d_model // self.num_heads 

        self.wq = torch.nn.Linear(d_model, d_model)
        self.wk = torch.nn.Linear(d_model, d_model)
        self.wv = torch.nn.Linear(d_model, d_model)

        self.final_linear = torch.nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):  # x [b, seq_len, d_model]
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = x.view(batch_size, -1, self.num_heads,
                   self.depth)  # [b, seq_len, d_model]=>[b, seq_len, num_head, depth]
        return x.transpose(1, 2)  # [b, seq_len, num_head, depth]=>[b, num_head, seq_len, depth]

    def forward(self, q, k, v, mask):  # q=k=v=x [b, seq_len, embedding_dim] embedding_dim = d_model
        batch_size = q.shape[0]

        q = self.wq(q)  # =>[batch_size, seq_len, d_model]
        k = self.wk(k)  # =>[batch_size, seq_len, d_model]
        v = self.wq(v)  # =>[batch_sizeb, seq_len, d_model]

        q = self.split_heads(q, batch_size)  # =>[batch_size, num_head, seq_len, depth]
        k = self.split_heads(k, batch_size)  # =>[batch_size, num_head, seq_len, depth]
        v = self.split_heads(v, batch_size)  # =>[batch_size, num_head, seq_len, depth]
        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        # => [batch_size, num_head, seq_len_q, depth], [batch_size, num_head, seq_len_q, seq_len_k]

        scaled_attention = scaled_attention.transpose(1, 2)  # =>[batch_size, seq_len_q, num_head, depth]
        concatenate_attention = scaled_attention.reshape(batch_size, -1, self.d_model)  # =>[batch_size, seq_len_q, d_model]

        output = self.final_linear(concatenate_attention)  # =>[batch_size, seq_len_q, d_model]
        return output, attention_weights  # [batch_size, seq_len_q, d_model], [batch_size, num_head, seq_len_q, seq_len_k]


## Point wise feed forward network

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
    feed_forward_net = torch.nn.Sequential(
        torch.nn.Linear(d_model, dff),  # [batch_size, seq_len, d_model]=>[batch_size, seq_len, dff]
        torch.nn.ReLU(),
        torch.nn.Linear(dff, d_model),  # [batch_size, seq_len, dff=2048]=>[batch_size, seq_len, d_model]
    )
    return feed_forward_net

## Encoder and Decoder

In [ ]:
class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)  # MultiHeadAttention（padding mask）(self-attention)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)
        self.layernorm2 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)

        self.dropout1 = torch.nn.Dropout(rate)
        self.dropout2 = torch.nn.Dropout(rate)

    # x [batch_size, inp_seq_len, embedding_dim] embedding_dim =d_model
    # mask [batch_size,1,1,inp_seq_len]
    def forward(self, x, mask):
        attn_output, _ = self.mha(x, x, x, mask)  # =>[batch_size, seq_len, d_model]
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)  # =>[batch_size, seq_len, d_model]

        ffn_output = self.ffn(out1)  # =>[batch_size, seq_len, d_model]
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  #  =>[batch_size, seq_len, d_model]

        return out2  # [batch_size, seq_len, d_model]

In [ ]:
class DecoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model,
                                       num_heads)  # MultiHeadAttention（look ahead mask and padding mask）(self-attention)
        self.mha2 = MultiHeadAttention(d_model, num_heads)  # MultiHeadAttention（padding mask）(encoder-decoder attention)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)
        self.layernorm2 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)
        self.layernorm3 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)

        self.dropout1 = torch.nn.Dropout(rate)
        self.dropout2 = torch.nn.Dropout(rate)
        self.dropout3 = torch.nn.Dropout(rate)

    # x [batch_size, targ_seq_len, embedding_dim] embedding_dim =d_model
    # look_ahead_mask [batch_size, 1, targ_seq_len, targ_seq_len] 
    # enc_output [batch_size, inp_seq_len, d_model]
    # padding_mask [batch_size, 1, 1, inp_seq_len]
    def forward(self, x, enc_output, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x,
                                               look_ahead_mask)  # =>[batch_size, targ_seq_len, d_model], [b, num_heads, targ_seq_len, targ_seq_len]
        attn1 = self.dropout1(attn1)
        out1 = self.layernorm1(x + attn1)  #  [batch_size, targ_seq_len, d_model]

        # Q: receives the output from decoder's first attention block，即 masked multi-head attention sublayer
        # K V: V (value) and K (key) receive the encoder output as inputs
        attn2, attn_weights_block2 = self.mha2(out1, enc_output, enc_output,
                                               padding_mask)  # =>[batch_size, targ_seq_len, d_model], [b, num_heads, targ_seq_len, inp_seq_len]
        attn2 = self.dropout2(attn2)
        out2 = self.layernorm2(out1 + attn2)  #  [batch_size, targ_seq_len, d_model]

        ffn_output = self.ffn(out2)  # =>[batch_size, targ_seq_len, d_model]
        ffn_output = self.dropout3(ffn_output)
        out3 = self.layernorm3(out2 + ffn_output)  # =>[batch_size, targ_seq_len, d_model]

        return out3, attn_weights_block1, attn_weights_block2
        # [batch_size, targ_seq_len, d_model], [batch_size, num_heads, targ_seq_len, targ_seq_len], [batch_size, num_heads, targ_seq_len, inp_seq_len]



In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self,
                 num_layers,  
                 d_model,
                 num_heads,
                 dff,  
                 pre_train,
                 input_vocab_size, 
                 maximun_position_encoding,
                 rate=0.1):
        super(Encoder, self).__init__()

        self.num_layers = num_layers
        self.d_model = d_model
        self.embedding = nn.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(1000,
                                                d_model)  # =>[1, max_pos_encoding, d_model=512]
        self.enc_layers = torch.nn.ModuleList([EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)])

        self.dropout = torch.nn.Dropout(rate)

    # x [batch_size, inp_seq_len]
    # mask [batch_size, 1, 1, inp_sel_len]
    def forward(self, x, mask):
        inp_seq_len = x.shape[-1]

        # adding embedding and position encoding
        x = self.embedding(x)  # [batch_size, inp_seq_len]=>[batch_size, inp_seq_len, d_model]
        x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        pos_encoding = self.pos_encoding[:, :inp_seq_len, :]
        pos_encoding = pos_encoding.cuda()  
        x += pos_encoding  # [batch_size, inp_seq_len, d_model]

        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, mask)  # [batch_size, inp_seq_len, d_model]=>[batch_size, inp_seq_len, d_model]
        return x  # [batch_size, inp_seq_len, d_model]

In [ ]:
class Decoder(torch.nn.Module):
    def __init__(self,
                 num_layers,  # N个encoder layer
                 d_model,
                 num_heads,
                 dff,  # dimension of ffn
                 target_vocab_size,  # target vocab size
                 maximun_position_encoding,
                 rate=0.1):
        super(Decoder, self).__init__()

        self.num_layers = num_layers
        self.d_model = d_model

        self.embedding = torch.nn.Embedding(num_embeddings=target_vocab_size, embedding_dim=d_model)
        self.pos_encoding = positional_encoding(1000,
                                                d_model)  # =>[1, max_pos_encoding, d_model=512]

        # self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate).cuda() for _ in range(num_layers)] # 不行
        self.dec_layers = torch.nn.ModuleList([DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)])

        self.dropout = torch.nn.Dropout(rate)

    # x [b, targ_seq_len]
    # look_ahead_mask [batch_size, 1, targ_seq_len, targ_seq_len] 
    # enc_output [batch_size, inp_seq_len, d_model]
    # padding_mask [batch_size, 1, 1, inp_seq_len]
    def forward(self, x, enc_output, look_ahead_mask, padding_mask):
        targ_seq_len = x.shape[-1]

        attention_weights = {}

        # adding embedding and position encoding
        x = self.embedding(x)  # [batch_size, targ_seq_len]=>[b, targ_seq_len, d_model]
        x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        pos_encoding = self.pos_encoding[:, :targ_seq_len, :]  # [b, targ_seq_len, d_model]
        pos_encoding = pos_encoding.cuda() #
        x += pos_encoding  # [batch_size, inp_seq_len, d_model]

        x = self.dropout(x)

        for i in range(self.num_layers):
            x, attn_block1, attn_block2 = self.dec_layers[i](x, enc_output, look_ahead_mask, padding_mask)
            # => [batch_size, targ_seq_len, d_model], [batch_size, num_heads, targ_seq_len, targ_seq_len], [batch_size, num_heads, targ_seq_len, inp_seq_len]

            attention_weights[f'decoder_layer{i + 1}_block1'] = attn_block1
            attention_weights[f'decoder_layer{i + 1}_block2'] = attn_block2

        return x, attention_weights

In [ ]:
loss_object = torch.nn.CrossEntropyLoss(reduction='none')
# real [batch_size, targ_seq_len]
# pred [batch_size, targ_seq_len, target_vocab_size]
def mask_loss_func(real, pred):
    # print(real.shape, pred.shape)
    # _loss = loss_object(pred, real) # [batch_size, targ_seq_len]
    _loss = loss_object(pred.transpose(-1, -2), real)  # [batch_size, targ_seq_len]
    mask = torch.logical_not(real.eq(pad)).type(_loss.dtype)  # [batch_size, targ_seq_len] 
    _loss *= mask
    return _loss.sum() / mask.sum().item()
def mask_accuracy_func(real, pred):
    _pred = pred.argmax(dim=-1)  # [batch_size, targ_seq_len, target_vocab_size]=>[b, targ_seq_len]
    corrects = _pred.eq(real)  # [batch_size, targ_seq_len] bool value
    mask = torch.logical_not(real.eq(pad))  # [batch_size, targ_seq_len] where index != 1 
    corrects *= mask
    return corrects.sum().float() / mask.sum().item()

In [ ]:
class Transformer(torch.nn.Module):
    def __init__(self,
                 num_layers,  # N个encoder layer
                 d_model,
                 num_heads,
                 dff,  
                 input_vocab_size,  
                 target_vocab_size,  
                 pre_train,
                 pe_input,  # input max_pos_encoding
                 pe_target,
                 # input max_pos_encoding
                 rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers,
                               d_model,
                               num_heads,
                               dff,pre_train,
                               input_vocab_size,
                               pe_input,
                               rate)
        self.decoder = Decoder(num_layers,
                               d_model,
                               num_heads,
                               dff,
                               target_vocab_size,
                               pe_target,
                               rate)
        self.final_layer = torch.nn.Linear(d_model, target_vocab_size)

    # inp [batch_size, inp_seq_len]
    # targ [batch_size, targ_seq_len]
    # enc_padding_mask [batch_size, 1, 1, inp_seq_len]
    # look_ahead_mask [batch_size, 1, targ_seq_len, targ_seq_len]
    # dec_padding_mask [batch_size, 1, 1, inp_seq_len] #
    def forward(self, inp, targ, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, enc_padding_mask)  # =>[batch_size, inp_seq_len, d_model]
        dec_output, attention_weights = self.decoder(targ, enc_output, look_ahead_mask, dec_padding_mask)
        # => [batch_size, targ_seq_len, d_model],
        # {'..block1': [batch_size, num_heads, targ_seq_len, targ_seq_len],
        #  '..block2': [batch_size, num_heads, targ_seq_len, inp_seq_len], ...}
        final_output = self.final_layer(dec_output)  # =>[b, targ_seq_len, target_vocab_size]

        return final_output, attention_weights


## Training

In [ ]:
def train_step(model, inp, targ):
    # tar_inp as input to decoder due to teacher force
    # tar_real is shift 1 prosition of in postion of tar_inp，tar_real include the groudtruth of next predicted token
    targ_inp = targ[:, :-1]
    targ_real = targ[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_mask(inp, targ_inp)

    inp = inp.to(device)
    targ_inp = targ_inp.to(device)
    targ_real = targ_real.to(device)
    enc_padding_mask = enc_padding_mask.to(device)
    combined_mask = combined_mask.to(device)
    dec_padding_mask = dec_padding_mask.to(device)
    # print('device:', inp.device, targ_inp)

    model.train()  # set train mode

    optimizer.zero_grad()  

    # forward
    prediction, _ = model(inp, targ_inp, enc_padding_mask, combined_mask, dec_padding_mask)
    loss = mask_loss_func(targ_real, prediction)
    metric = mask_accuracy_func(targ_real, prediction)

    # backward
    loss.backward()  
    optimizer.step()  

    return loss.item(), metric.item()

In [ ]:
def validate_step(model, inp, targ):
    targ_inp = targ[:, :-1]
    targ_real = targ[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_mask(inp, targ_inp)

    inp = inp.to(device)
    targ_inp = targ_inp.to(device)
    targ_real = targ_real.to(device)
    enc_padding_mask = enc_padding_mask.to(device)
    combined_mask = combined_mask.to(device)
    dec_padding_mask = dec_padding_mask.to(device)

    model.eval()  # set eval mode

    with torch.no_grad():
        # forward
        prediction, _ = model(inp, targ_inp, enc_padding_mask, combined_mask, dec_padding_mask)
        val_loss = mask_loss_func(targ_real, prediction)
        val_metric = mask_accuracy_func(targ_real, prediction)

    return val_loss.item(), val_metric.item()


In [ ]:
EPOCHS = 20 # 50 # 30  # 20

print_trainstep_every = 50  # 每50个step做一次打印

metric_name = 'acc'

def printbar():
    nowtime = datetime.datetime.now().strftime('%Y-%m_%d %H:%M:%S')
    print('\n' + "=========="*8 + '%s'%nowtime)


In [ ]:
# training function
def train_model(model, epochs,ds_train, ds_val ,ds_test,ds_test1,ds_test2, print_every,BATCH_SIZE):
    starttime = time.time()
    print('*' * 27, 'start training...')
    printbar()

    best_acc = 0.
    # initialize record frame work
    df_history = pd.DataFrame(columns=['epoch', 'loss', metric_name, 'val_loss', 'val_' + metric_name,'test_loss', 'test_' + metric_name, 'test1_loss', 'test1_' + metric_name,'test2_loss', 'test2_' + metric_name])
    for epoch in range(1,epochs+1):
        # laod data
        train_iter, val_iter,test_iter,test_iter1,test_iter2 = torchtext.legacy.data.Iterator.splits(
    (ds_train, ds_val,ds_test,ds_test1,ds_test2),
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    batch_sizes=(BATCH_SIZE, 256,256,256,256),
    shuffle = True
)
        val_dataloader = DataLoader(val_iter)
        test_dataloader = DataLoader(test_iter) 
        test_dataloader1 = DataLoader(test_iter1)
        test_dataloader2 = DataLoader(test_iter2)
        train_dataloader = DataLoader(train_iter)
        loss_sum = 0.
        metric_sum = 0.

        for step, (inp, targ) in enumerate(train_dataloader,start = 1):
            # inp [64, 10] , targ [64, 10]
            loss, metric = train_step(model, inp, targ)

            loss_sum += loss
            metric_sum += metric
            if step % print_every == 0:
                print('*' * 8, f'[step = {step}] loss: {loss_sum / step:.3f}, {metric_name}: {metric_sum / step:.3f}')

        # adjust lr
        scheduler.step()
        val_loss_sum = 0.
        val_metric_sum = 0.
        for val_step, (inp, targ) in enumerate(val_dataloader, start=1):

            loss, metric = validate_step(model, inp, targ)

            val_loss_sum += loss
            val_metric_sum += metric

        test_loss_sum = 0.
        test_metric_sum = 0.
        for test_step, (inp, targ) in enumerate(test_dataloader, start=1):

            loss, metric = validate_step(model, inp, targ)
            test_loss_sum += loss
            test_metric_sum += metric
        test_loss_sum1 = 0.
        test_metric_sum1 = 0.
        for test_step1, (inp, targ) in enumerate(test_dataloader1, start=1):
            loss, metric = validate_step(model, inp, targ)

            test_loss_sum1 += loss
            test_metric_sum1 += metric
        test_loss_sum2 = 0.
        test_metric_sum2 = 0.
        bleu = []
        for test_step2, (inp, targ) in enumerate(test_dataloader2, start=1):
            loss, metric = validate_step(model, inp, targ)

            test_loss_sum2 += loss
            test_metric_sum2 += metric
        if epoch<1000:
          record = (epoch, loss_sum/step, metric_sum/step, val_loss_sum/val_step, val_metric_sum/val_step,
                  test_loss_sum/test_step, test_metric_sum/test_step,test_loss_sum1/test_step1, test_metric_sum1/test_step1,test_loss_sum2/test_step2, test_metric_sum2/test_step2)
          df_history.loc[epoch - 1] = record
          print('EPOCH = {} loss: {:.3f}, {}: {:.3f}, val_loss: {:.3f}, val_{}: {:.3f},test_loss: {:.3f}, test_{}: {:.3f}, test1_loss: {:.3f}, test1_{}: {:.3f},test2_loss: {:.3f}, test2_{}: {:.3f}'.format(
            record[0], record[1], metric_name, record[2], record[3], metric_name, record[4],record[5],metric_name, record[6],record[7],metric_name,record[8],record[9],metric_name,record[10]))
          printbar()
          # early stop
          if len(df_history)>=11:
            if record[3]>=df_history.loc[epoch - 11][3]:
              break
          else:
            continue
    print('finishing training...')
    endtime = time.time()
    time_elapsed = endtime - starttime
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return df_history



## Generating FOL

In [ ]:
def tokenizer_encode(tokenize, sentence, vocab):
    # print(type(vocab)) # torchtext.vocab.Vocab
    # print(len(vocab))
    sentence =sentence.lower()
    # print(type(sentence)) # str
    sentence = tokenize(sentence)  # list
    sentence = ['sos'] + sentence + ['eos']
    sentence_ids = [vocab.stoi[token] for token in sentence]
    # print(sentence_ids, type(sentence_ids[0])) # int
    return sentence_ids



def tokenzier_decode1(sentence_ids, vocab):
    sentence = [vocab.itos[id] for id in sentence_ids if id<len(vocab)]
    # print(sentence)
    return " ".join(sentence[1:])

def tokenzier_decode_char(sentence_ids, vocab):
    sentence = [vocab.itos[id] for id in sentence_ids if id<len(vocab)]
    # print(sentence)
    return "".join(sentence[1:])

def evaluate(model, inp_sentence):
    model.eval()  # 设置eval mode

    inp_sentence_ids = tokenizer_encode(tokenizer, inp_sentence, SRC_TEXT.vocab)  # 转化为索引
    # print(tokenzier_decode(inp_sentence_ids, SRC_TEXT.vocab))
    encoder_input = torch.tensor(inp_sentence_ids).unsqueeze(dim=0)  # =>[batch_size=1, inp_seq_len=10]
    # print(encoder_input.shape)
    decoder_input = [TARG_TEXT.vocab.stoi['sos']]
    decoder_input = torch.tensor(decoder_input).unsqueeze(0)  # =>[batch_size=1,seq_len=1]
    # print(decoder_input.shape)

    with torch.no_grad():
        enc_padding_mask, combined_mask, dec_padding_mask = create_mask(encoder_input.cpu(), decoder_input.cpu()) ################
        encoder_input = encoder_input.to(device)
        enc_padding_mask = enc_padding_mask.to(device)
        enout = model.encoder(encoder_input, enc_padding_mask)
        for i in range(200 + 2):
            enc_padding_mask, combined_mask, dec_padding_mask = create_mask(encoder_input.cpu(), decoder_input.cpu()) ################
            decoder_input = decoder_input.to(device)
            combined_mask = combined_mask.to(device)
            dec_padding_mask = dec_padding_mask.to(device)
            predictions, attention_weights = model.decoder(decoder_input, enout, combined_mask, dec_padding_mask)
            predictions = model.final_layer(predictions)
            prediction = predictions[:, -1:, :]  # =>[b=1, 1, target_vocab_size]
            prediction_id = torch.argmax(prediction, dim=-1)  # => [b=1, 1]
            if prediction_id.squeeze().item() == TARG_TEXT.vocab.stoi['eos']:
                return decoder_input.squeeze(dim=0), attention_weights

            decoder_input = torch.cat([decoder_input, prediction_id],
                                      dim=-1)  # [b=1,targ_seq_len=1]=>[b=1,targ_seq_len=2]
            # decoder_input being longer
    return decoder_input.squeeze(dim=0), attention_weights
    # [targ_seq_len],


In [ ]:
num_layers = 3
d_model =300
dff = 600
num_heads = 5

input_vocab_size = len(SRC_TEXT.vocab) # 3901
target_vocab_size = len(TARG_TEXT.vocab) # 2591
dropout_rate = 0.3
torch.cuda.empty_cache()
transformer = Transformer(num_layers,
                          d_model,
                          num_heads,
                          dff,
                          input_vocab_size,
                          target_vocab_size,
                          None,
                          pe_input=1000,#set 1000 if char-level
                          pe_target=1000,#set 1000 if char-level
                          
                          rate=dropout_rate)
transformer = transformer.to(device)


optimizer = torch.optim.Adam(transformer.parameters(),lr=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer , 5,
                gamma =0.9)
df_history = train_model(transformer, 150, ds_train, ds_val, ds_test,ds_test1,ds_test2,print_trainstep_every,256)

*************************** start training...

================================================================================2021-09_27 08:11:25
******** [step = 50] loss: 2.001, acc: 0.454
******** [step = 100] loss: 1.747, acc: 0.508
******** [step = 150] loss: 1.596, acc: 0.542
******** [step = 200] loss: 1.489, acc: 0.565
EPOCH = 1 loss: 1.484, acc: 0.566, val_loss: 1.073, val_acc: 0.660,test_loss: 1.079, test_acc: 0.658, test1_loss: 1.106, test1_acc: 0.653,test2_loss: 1.130, test2_acc: 0.648

================================================================================2021-09_27 08:12:34
******** [step = 50] loss: 1.106, acc: 0.652
******** [step = 100] loss: 1.075, acc: 0.660
******** [step = 150] loss: 1.052, acc: 0.666
******** [step = 200] loss: 1.032, acc: 0.671
EPOCH = 2 loss: 1.031, acc: 0.672, val_loss: 0.899, val_acc: 0.708,test_loss: 0.906, test_acc: 0.707, test1_loss: 0.945, test1_acc: 0.699,test2_loss: 0.980, test2_acc: 0.692

=====================================

In [2]:

# gt= [[i[1].split()]for i in test_pairs]
# from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import SmoothingFunction
# smoothie = SmoothingFunction().method4
# total = 0
# for i in tqdm(range(len(test_pairs))):
       
      
#         references = gt[i]
      
#         pred_result, _ = evaluate(transformer, test_pairs[i][0])
#         candidates = tokenzier_decode1(pred_result, TARG_TEXT.vocab).split() 
#         # candidates = tokenzier_decode_char(pred_result, TARG_TEXT.vocab).split() if char-level

#         total+=sentence_bleu(references, candidates,smoothing_function=smoothie)
# total*100/len(test_pairs)

In [1]:

# gt= [[i[1].split()]for i in test_pairs1]
# from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import SmoothingFunction
# smoothie = SmoothingFunction().method4
# total = 0
# for i in tqdm(range(len(test_pairs1))):
       
      
#         references = gt[i]
      
#         pred_result, _ = evaluate(transformer, test_pairs1[i][0])
#         candidates = tokenzier_decode1(pred_result, TARG_TEXT.vocab).split()
#         # candidates = tokenzier_decode_char(pred_result, TARG_TEXT.vocab).split() if char-level

#         total+=sentence_bleu(references, candidates,smoothing_function=smoothie)
# total*100/len(test_pairs1)


In [3]:

# gt= [[i[1].split()]for i in test_pairs2]
# from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import SmoothingFunction
# smoothie = SmoothingFunction().method4
# total = 0
# for i in tqdm(range(len(test_pairs2))):
       
      
#         references = gt[i]
      
#         pred_result, _ = evaluate(transformer, test_pairs2[i][0])
#         candidates = tokenzier_decode1(pred_result, TARG_TEXT.vocab).split()
#         # candidates = tokenzier_decode_char(pred_result, TARG_TEXT.vocab).split() if char-level

#         total+=sentence_bleu(references, candidates,smoothing_function=smoothie)
# total*100/len(test_pairs2)

In [ ]:
def batch_translate(pairs,n):
    for i in range(n):
        pair = random.choice(pairs)
        print('input:', pair[0])
        print('target:', pair[1])
        pred_result, _ = evaluate(transformer, pair[0])
        # pred_sentence = tokenzier_decode_char(pred_result, TARG_TEXT.vocab) if char-level
        pred_sentence = tokenzier_decode1(pred_result, TARG_TEXT.vocab)
        print('pred:', pred_sentence)
        print('')


In [ ]:
batch_translate(test_pairs,10)